In [1]:
import importlib
import pandas as pd
import odqa_encoder
import odqa_milvus
import odqa_mysql
from odqa_answer_extractor import answer_extract


Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.bias', 'predictions.dense.weight', 'sop_classifier.classifier.weight', 'predictions.decoder.weight', 'sop_classifier.classifier.bias', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


odqa


In [2]:
import json

In [3]:
def push_context_to_milvus():

    # db_fp = r"database_handler.json"
    # file = open(db_fp)
    # database_handler = json.loads(file.read())
    # file.close()

    start= database_handler['milvus_rows']
    end= start+database_handler["batch"]
    index = database_handler['milvus_rows']
    
    query = f"select * from context where id between {start} and {end} ;"
    res = odqa_mysql.execute_query(query)

    for id, context in res:
        emb = odqa_encoder.encode(context)
        collection.insert([[index], [id], emb])
        index+=1  
           
    database_handler['milvus_rows'] = end
    database_handler['milvus_index'] = index

    file = open(db_fp,"w")
    json.dump(database_handler, file)
    file.close()
    

In [4]:
question = ["மனித உடலில் எத்தனை எலும்புகள் உள்ளன?"]


In [5]:
question_emb = odqa_encoder.encode(question)

similar_ids = odqa_milvus.find_similar(question_emb)
sim_ids = similar_ids[0].ids
retrieved_context = odqa_mysql.extract_context(sim_ids[0])

AttributeError: 'NoneType' object has no attribute 'load'

In [6]:
QA_input = {
    'question': question,
    'context': retrieved_context
}
res = answer_extract(QA_input)
res

[{'score': 3.443395598878851e-06,
  'start': 507,
  'end': 514,
  'answer': ' நான்கு'}]

In [10]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
connections.connect()
TABLE_NAME = 'question_answering'

In [15]:
field1 = FieldSchema(name="ind", dtype=DataType.INT64, descrition="int64", is_primary=True)
field2 = FieldSchema(name="id", dtype=DataType.INT64, descrition="int64", is_primary=False)
field3 = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, descrition="float vector",dim=768, is_primary=False)
schema = CollectionSchema(fields=[field1, field2, field3], description="collection description")
collection = Collection(name=TABLE_NAME, schema=schema)
default_index = {"index_type": "IVF_FLAT", "metric_type": 'IP', "params": {"nlist": 200}}
collection.create_index(field_name="embedding", index_params=default_index)

Status(code=0, message='')

In [14]:
if utility.has_collection(TABLE_NAME):
    collection = Collection(name=TABLE_NAME)
    collection.drop()
    
    # print(collection)

In [16]:
collection

<Collection>:
-------------
<name>: question_answering
<partitions>: [{"name": "_default", "collection_name": "question_answering", "description": ""}]
<description>: collection description
<schema>: {
  auto_id: False
  description: collection description
  fields: [{
    name: ind
    description: 
    type: 5
    is_primary: True
    auto_id: False
  }, {
    name: id
    description: 
    type: 5
  }, {
    name: embedding
    description: 
    type: 101
    params: {'dim': 768}
  }]
}

In [17]:
search_params = {"metric_type": "IP", "params": {"nprobe": 10}}


In [5]:
def find_similar(emb):
    collection.load()
    return collection.search(
	data=emb, 
	anns_field="embedding", 
	param=search_params, 
	limit=10, 
	expr=None,
	consistency_level="Strong"
)

# encoder 

In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

import torch
import torch.nn.functional as F

#Mean Pooling - Take average of all tokens
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
#Encode text
def encode(texts):
    # Tokenize sentences
    doc_stride = 128
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512, stride=doc_stride, return_overflowing_tokens = True)
    encoded_input.pop("overflow_to_sample_mapping")

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input, return_dict=True)

    # Perform pooling
    embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    embeddings = F.normalize(embeddings, p=2, dim=1)
    
    return embeddings.tolist()

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.decoder.weight', 'predictions.dense.weight', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'sop_classifier.classifier.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
index, id = 0, 0
context = ["ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக, மிகக் குறைந்த எண்ணிக்கையான மனிதர்களில், ஒரு மேலதிக விலா எலும்பு (கழுத்துவில்) அல்லது ஒரு மேலதிகமான கீழ் முதுகெலும்பு காணப்படுவதுண்டு; இணைந்த சில எலும்புகளைத் தனி எலும்பாகக் கருதாவிடின், ஐந்து இணைந்த திருவெலும்பு; மூன்று (3 - 5) குயிலலகு எலும்புகள் சேர்ந்து 26 எண்ணிக்கையிலான முதுகெலும்புகள் 33 ஆகக் கருதப்படலாம்.\nமனித மண்டையோட்டில் 22 எலும்புகள் (காதுச் சிற்றென்புகளைத் தவிர) உள்ளன; இவை எட்டு மண்டையறை (cranium) எலும்புகளாகவும் 14 முக எலும்புகளாகவும் (facial bones) பிரிக்கப்பட்டுள்ளன. (தடித்த எண்கள் அருகிலுள்ள படத்தில் காணும் எண்களைக் குறிக்கின்றன.)\n\n\n\nமண்டையறை எலும்புகள் (8)\n 1 நுதலெலும்பு (frontal bone) \n 2 சுவரெலும்பு (parietal bone) (2)\n 3 கடைநுதலெலும்பு (temporal bone) (2)\n 4 பிடர் எலும்பு (occipital bone)\n ஆப்புரு எலும்பு (sphenoid bone)\n நெய்யரியெலும்பு (ethmoid bone)\nமுக எலும்புகள் (14)\n 7 கீழ்த்தாடை எலும்பு (mandible)\n 6 மேற்றாடை எலும்பு (maxilla) (2)\n அண்ணவெலும்பு (palatine bone) (2)\n 5 கன்ன எலும்பு (zygomatic bone) (2)\n 9 நாசி எலும்பு (nasal bone) (2)\n கண்ணீர் எலும்பு (lacrimal bone) (2)\n மூக்குச் சுவர் எலும்பு (vomer)\n கீழ் மூக்குத் தடுப்பெலும்பு (inferior nasal conchae) (2)\nநடுக்காதுகளில் (6):\n சம்மட்டியுரு (malleus)\n பட்டையுரு (incus) \n ஏந்தியுரு (stapes)\nதொண்டையில் (1):\n தொண்டை எலும்பு (நாவடி எலும்பு) (hyoid)\nதோள் பட்டையில் (4):\n 25. காறை எலும்பு (clavicle)\n 29. தோள் எலும்பு (scapula)\nமார்புக்கூட்டில் thorax(25): \n 10. மார்பெலும்பு (sternum) (1)\n மேலும் மூன்று என்புகளாகக் கருதப்படலாம்: பிடியுரு (manubrium), உடல் மார்பெலும்பு (body of sternum), வாள்வடிவ நீட்டம் (xiphoid process)\n 28. விலா எலும்புகள் (rib) (24)\n முதுகெலும்புத் தூண் (vertebral column) (33): \n 8. கழுத்து முள்ளெலும்புகள் (cervical vertebra) (7)\n மார்பு முள்ளெலும்புகள் (thoracic vertebra) (12)\n 14. நாரிமுள்ளெலும்புகள் (lumbar vertebra) (5)\n 16. திரிகம் (திருவெலும்பு) (sacrum)\n வால் எலும்பு (குயிலலகு) (coccyx)\nமேற்கைகளில் (arm) (1): \n 11. புய எலும்பு (மேல்கை எலும்பு) (humerus)\n 26. புய எலும்புப் புடைப்பு (மேல்கை எலும்புப் புடைப்பு) (condyles of humerus)\nமுன்கைகளில் (forearm) (4): \n 12. அரந்தி (ulna) (2)\n 13. ஆரை எலும்பு (radius) (2)\n 27. ஆரை எலும்புத் தலை (head of radius)\nகைகளில் (hand) (54): \n மணிக்கட்டுகள் (carpal):\n படகெலும்பு (scaphoid) (2)\n பிறைக்குழி எலும்பு (lunate) (2)\n முப்பட்டை எலும்புtriquetrum) (2)\n பட்டாணி எலும்பு (pisiform) (2)\n சரிவக எலும்பு (trapezium) (2)\n நாற்புறவுரு எலும்பு (trapezoid) (2)\n தலையுரு எலும்பு (capitate) (2)\n கொக்கி எலும்பு (hamate) (2)\n அங்கை முன்னெலும்புகள் (அனுமணிக்கட்டு எலும்புகள்) (metacarpal): (5 × 2)\n விரலெலும்புகள் (phalange): \n அண்மை விரலெலும்புகள் (proximal phalanges) (5 × 2)\n நடு விரலெலும்புகள் (Intermediate phalanges) (4 × 2)\n தொலை விரலெலும்புகள் (distal phalanges) (5 × 2)\nஇடுப்பு வளையம் (pelvis) (2):\n 15. இடுப்பெலும்பு (ilium) மற்றும் கீழ் இடுப்பெலும்பு (ischium)\n\n\nகால்கள் (leg) (8): \n 18. தொடையெலும்பு (femur) (2)\n 17. இடுப்பு மூட்டு (hip joint) (மூட்டு, எலும்பல்ல) \n 22. பெரிய தொடையெலும்புக் கொண்டை (greater trochanter of femur)\n 23. தொடையெலும்புப் புடைப்பு (condyles of femur)\n 19. சில்லெலும்பு (patella) (2)\n 20. கால் முன்னெலும்பு (கணைக்காலலுள்ளெலும்பு) (tibia) (2)\n 21. சிம்பு எலும்பு (கணைக்கால்வெளியெலும்பு) (fibula) (2)\nகாலடிகளில் (52): \n கணுக்காலெலும்புகள் (tarsal):\n குதிகால் (calcaneus) (2)\n முட்டி (talus) (2)\n படகுரு எலும்பு (navicular bone) (2)\n உள் ஆப்புவடிவ எலும்பு (2) \n இடை ஆப்புவடிவ எலும்பு (2)\n வெளி ஆப்புவடிவ எலும்பு (2)\n கனசதுர எலும்பு (cuboidal bone) (2)\n அனுகணுக்காலெலும்புகள் (metatarsal) (5 × 2)\n விரலெலும்புகள் (phalange):\n அண்மை விரலெலும்புகள் (proximal phalanges) (5 × 2)\n நடு விரலெலும்புகள் (intermediate phalanges) (4 × 2)\n தொலை விரலெலும்புகள் (distal phalanges) (5 × 2)\n குழந்தை எலும்புக்கூடு \nகுழந்தைகளின் எலும்புக்கூடுகளில் கீழ் வரும் எலும்புகள் மேலதிகமாக உள்ளன:\n மண்டையறை மற்றும் மண்டையோட்டு எலும்புகள் (21), இவை ஒன்றாகி மண்டையறையை உருவாக்குகின்றன.\n திரிக முள்ளெலும்புகள் (sacral vertebrae) (4 or 5), வளர்ந்தோரில் இவை ஒன்றாகி திரிகத்தை உருவாக்குகின்றன \n coccygeal vertebrae (3 to 5), வளர்ந்தோரில் இவை ஒன்றாகி வாலெலும்பை உருவாக்குகின்றன\n இடுப்பெலும்பு, கீழ் இடுப்பெலும்பு மற்றும் பொச்செலும்பு (pubis), என்பவை வளர்ந்தோரில் ஒன்றாகி இடுப்பெலும்பை உருவாக்குகின்றன\n\nபகுப்பு:மனித உடற்கூற்றியல்\nபகுப்பு:எலும்புகள்"]
res = [[id, context]]

In [ ]:
for id, context in res:
        emb = encode(context)
        indexs = []
        ids = []
        for i in range(len(emb)):
            indexs.append(index)
            index+=1  
            ids.append(id)
        print(emb, indexs, ids)